In [25]:
%pip install statsmodels


Note: you may need to restart the kernel to use updated packages.


# Alternativas para calcular tamaño de muestra en Python

Comparemos diferentes métodos y librerías para calcular el tamaño de muestra necesario para intervalos de confianza de proporciones.

In [26]:
# Método 1: Fórmula manual con scipy.stats (el que ya tienes)
from scipy.stats import norm

confidence_level = 0.95
margin_error = 0.03
p_hat = 0.5

z_score = norm.ppf(1 - (1 - confidence_level) / 2)
n_manual = (z_score / margin_error) ** 2 * p_hat * (1 - p_hat)

print(f"Método 1 (manual con scipy): {n_manual:.4f} ≈ {round(n_manual)}")

Método 1 (manual con scipy): 1067.0719 ≈ 1067


In [27]:
from statsmodels.stats.proportion import samplesize_confint_proportion

# Params
alpha = 1 - confidence_level  # 0.05
half_width = margin_error     # 0.03

n_statsmodels = samplesize_confint_proportion(p_hat, half_width, alpha=alpha)

print(f"Método 2 (statsmodels): {n_statsmodels:.4f} ≈ {round(n_statsmodels)}")

Método 2 (statsmodels): 1067.0719 ≈ 1067


In [ ]:
# Params
alpha = 0.05   # 0.05
half_width = margin_error     # 0.03

n_statsmodels = samplesize_confint_proportion(p_hat, half_width, alpha=alpha)

print(f"Método 2 (statsmodels): {n_statsmodels:.4f} ≈ {round(n_statsmodels)}")

Método 2 (statsmodels): 1843.0268 ≈ 1843


# Cálculo de Margen de Error con diferentes librerías

Ahora veamos cómo calcular el **margen de error** dado un nivel de confianza, tamaño de muestra y proporción estimada usando diferentes librerías de Python.

In [5]:
# Método 1: Cálculo manual del margen de error con scipy.stats
from scipy.stats import norm
import math

# Parámetros
confidence_level = 0.95
n_sample = 232  # tamaño de muestra
p_hat = 0.5      # proporción estimada

# Calcular el margen de error
z_score = norm.ppf(1 - (1 - confidence_level) / 2)
margin_error_manual = z_score * math.sqrt((p_hat * (1 - p_hat)) / n_sample)

print(f"Método 1 (manual con scipy):")
print(f"  Margen de error: {margin_error_manual:.6f}")
print(f"  Intervalo: [{p_hat - margin_error_manual:.4f}, {p_hat + margin_error_manual:.4f}]")
print()

Método 1 (manual con scipy):
  Margen de error: 0.064339
  Intervalo: [0.4357, 0.5643]



In [6]:
# Método 2: statsmodels - proportion_confint
from statsmodels.stats.proportion import proportion_confint

# Calcular intervalo de confianza directamente
n_successes = int(p_hat * n_sample)  # número de éxitos
ci_lower, ci_upper = proportion_confint(n_successes, n_sample, alpha=1-confidence_level, method='normal')

# El margen de error es la mitad del ancho del intervalo
margin_error_statsmodels = (ci_upper - ci_lower) / 2

print(f"Método 2 (statsmodels.proportion_confint):")
print(f"  Intervalo: [{ci_lower:.6f}, {ci_upper:.6f}]")
print(f"  Margen de error: {margin_error_statsmodels:.6f}")
print()

Método 2 (statsmodels.proportion_confint):
  Intervalo: [0.435661, 0.564339]
  Margen de error: 0.064339



In [7]:
# Método 3: scipy.stats con distribución t de Student (más preciso para muestras pequeñas)
from scipy.stats import t

# Para muestras grandes, t se aproxima a normal, pero es más preciso
degrees_freedom = n_sample - 1
t_score = t.ppf(1 - (1 - confidence_level) / 2, degrees_freedom)
margin_error_t = t_score * math.sqrt((p_hat * (1 - p_hat)) / n_sample)

print(f"Método 3 (scipy.stats con t-Student):")
print(f"  t-score: {t_score:.6f}")
print(f"  Margen de error: {margin_error_t:.6f}")
print(f"  Intervalo: [{p_hat - margin_error_t:.6f}, {p_hat + margin_error_t:.6f}]")
print()

Método 3 (scipy.stats con t-Student):
  t-score: 1.970287
  Margen de error: 0.064678
  Intervalo: [0.435322, 0.564678]



In [8]:
# Método 4: numpy con percentiles
import numpy as np

# Usando percentiles para encontrar el valor crítico
alpha = 1 - confidence_level
critical_value = np.abs(norm.ppf(alpha/2))
margin_error_numpy = critical_value * np.sqrt((p_hat * (1 - p_hat)) / n_sample)

print(f"Método 4 (numpy con percentiles):")
print(f"  Valor crítico: {critical_value:.6f}")
print(f"  Margen de error: {margin_error_numpy:.6f}")
print(f"  Intervalo: [{p_hat - margin_error_numpy:.6f}, {p_hat + margin_error_numpy:.6f}]")
print()

Método 4 (numpy con percentiles):
  Valor crítico: 1.959964
  Margen de error: 0.064339
  Intervalo: [0.435661, 0.564339]



In [ ]:
# Método 5: Con la librería pingouin (si está disponible)
try:
    import pingouin as pg
    
    # Simular datos binarios para el cálculo
    data = np.random.binomial(1, p_hat, n_sample)
    
    # Calcular intervalo de confianza
    ci = pg.ci(data, confidence=confidence_level)
    margin_error_pingouin = (ci[1] - ci[0]) / 2
    
    print(f"Método 5 (pingouin):")
    print(f"  Intervalo: [{ci[0]:.6f}, {ci[1]:.6f}]")
    print(f"  Margen de error: {margin_error_pingouin:.6f}")
    print()
    
except ImportError:
    print("Método 5: pingouin no está instalado")
    print("Para instalarlo: pip install pingouin")
    print()

In [ ]:
# Comparación de todos los métodos
print("="*60)
print("RESUMEN DE MÁRGENES DE ERROR:")
print("="*60)
print(f"Método 1 (scipy manual):     {margin_error_manual:.6f}")
print(f"Método 2 (statsmodels):      {margin_error_statsmodels:.6f}")
print(f"Método 3 (t-Student):        {margin_error_t:.6f}")
print(f"Método 4 (numpy):            {margin_error_numpy:.6f}")

# Mostrar diferencias
print("\nDiferencias respecto al método manual:")
print(f"statsmodels vs manual:       {abs(margin_error_statsmodels - margin_error_manual):.8f}")
print(f"t-Student vs manual:         {abs(margin_error_t - margin_error_manual):.8f}")
print(f"numpy vs manual:             {abs(margin_error_numpy - margin_error_manual):.8f}")

print(f"\nPara n={n_sample}, p={p_hat}, confianza={confidence_level*100}%")

In [ ]:
# Función práctica para calcular margen de error
def calcular_margen_error(n, p=0.5, confidence=0.95, metodo='normal'):
    """
    Calcula el margen de error para una proporción
    
    Parámetros:
    - n: tamaño de muestra
    - p: proporción estimada (default: 0.5 para máximo margen)
    - confidence: nivel de confianza (default: 0.95)
    - metodo: 'normal' o 't-student'
    
    Retorna:
    - margen de error
    """
    alpha = 1 - confidence
    
    if metodo == 'normal':
        critical_value = norm.ppf(1 - alpha/2)
    elif metodo == 't-student':
        critical_value = t.ppf(1 - alpha/2, n-1)
    else:
        raise ValueError("Método debe ser 'normal' o 't-student'")
    
    margin_error = critical_value * math.sqrt((p * (1 - p)) / n)
    return margin_error

# Ejemplos de uso
print("Función práctica - Ejemplos:")
print(f"n=100, confianza=95%:  {calcular_margen_error(100):.6f}")
print(f"n=500, confianza=95%:  {calcular_margen_error(500):.6f}")
print(f"n=1000, confianza=95%: {calcular_margen_error(1000):.6f}")
print(f"n=1000, confianza=99%: {calcular_margen_error(1000, confidence=0.99):.6f}")
print(f"n=1000, t-student:     {calcular_margen_error(1000, metodo='t-student'):.6f}")

In [10]:
# Método 1: Fórmula manual con scipy.stats (el que ya tienes)
from scipy.stats import norm

confidence_level = 0.98
margin_error = 0.03
p_hat = 0.5

z_score = norm.ppf(1 - (1 - confidence_level) / 2)
n_manual = (z_score / margin_error) ** 2 * p_hat * (1 - p_hat)

print(f"Método 1 (manual con scipy): {n_manual:.4f} ≈ {round(n_manual)}")

Método 1 (manual con scipy): 1503.3040 ≈ 1503
